### This Notebook Provides some Notes about using PyTorch for LogisticRegression

In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import torch
import torch.nn as nn

from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

### Load and Prepare the Data

In [2]:
## get the data
dataset = datasets.load_breast_cancer()
X = dataset['data']
y = dataset['target']

y = y.reshape(-1, 1)

## Split the Data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

## standardize the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


## to float32 then to tensors
X_train_scaled = torch.from_numpy(X_train_scaled.astype(np.float32))
X_test_scaled = torch.from_numpy(X_test_scaled.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.reshape(-1, 1)  ## --> reshape y
y_test = y_test.reshape(-1, 1)  ## --> reshape y

#### Build The Model

In [3]:
## Create a class for Logistic Regression
class LogisticRegression(nn.Module):
    def __init__(self, input_features):
        super(LogisticRegression, self).__init__()
        
        self.linear = nn.Linear(input_features, 1) ## --> 1 output class
    
    def forward(self, x):
        y_pred = torch.sigmoid(self.linear(x))
        return y_pred
    


## intance 
log_reg = LogisticRegression(input_features=X.shape[1])


## Training
print('Training started __________________________________ \n')

## criteria
learning_rate = 0.01
n_epochs = 100

loss = nn.BCELoss()   ## Binary cross entropy
optimizer = torch.optim.SGD(log_reg.parameters(), lr=learning_rate)

for epoch in range(n_epochs):
    ## predicting
    y_pred_train = log_reg(X_train_scaled)
    ## loss
    l = loss(y_pred_train, y_train)
    ## backpropagation
    l.backward()
    ## optimizaion step
    optimizer.step()
    ## empty the gradients
    optimizer.zero_grad()


print('Training finished ____________________________________ \n') 
y_pred_train_cls = y_pred_train.round()
train_acc = (y_pred_train_cls==y_train).sum() / y_train.shape[0]
print(f'Train Accuracy is ---> {train_acc * 100} %')

Training started __________________________________ 

Training finished ____________________________________ 

Train Accuracy is ---> 96.26373291015625 %


#### Final Evaluation

In [4]:
## During Evaluation you should stop the (requires_grad=True)
## you can use --> with torch.no_grad()

with torch.no_grad():
    ## predict for test with requires_grad=False(by using with)
    y_pred_test = log_reg(X_test_scaled)
    ## get the loss of the test
    loss_test = loss(y_pred_test, y_test)
    
    ## get the classes (0, 1) by rounding
    y_pred_test_cls = y_pred_test.round()
    test_acc = (y_pred_test_cls==y_test).sum() / y_test.shape[0]
    
    print(f'Test Accuracy is ---> {test_acc * 100} %')

Test Accuracy is ---> 95.6140365600586 %


### Done!